In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving EEG.machinelearing_data_BRMH.csv to EEG.machinelearing_data_BRMH.csv


# 3D Geodesic Dome as the Structural Metaphor for Link Theory

This visualization represents a geodesic dome-inspired brain network, where:

- **Each node** is a neuron or region.
- **Each edge** reflects a functional or structural connection.
- **Triangles** form the foundational motifs providing stability.

Geometrically, the dome allows for load distribution, redundancy, and predictable collapse points — mirroring how brains may reroute and fail under chronic stress.

This provides a compelling foundation for Link Theory, allowing us to map real EEG-derived networks into this shell to simulate real failures and rewiring within a brain-shaped topology.


In [ ]:
import numpy as np
import networkx as nx
import plotly.graph_objects as go
from scipy.spatial import Delaunay

# Generate dome coordinates using a hemisphere
phi = np.linspace(0, np.pi, 10)     # vertical
theta = np.linspace(0, 2 * np.pi, 20)  # horizontal
phi, theta = np.meshgrid(phi, theta)

x = np.sin(phi) * np.cos(theta)
y = np.sin(phi) * np.sin(theta)
z = np.cos(phi)

# Flatten coordinate arrays
points = np.vstack((x.flatten(), y.flatten(), z.flatten())).T

# Keep only upper hemisphere
points = points[points[:, 2] >= 0]

# Use Delaunay triangulation to connect nearby points
tri = Delaunay(points[:, :2])
edges = set()
for simplex in tri.simplices:
    for i in range(3):
        edge = tuple(sorted((simplex[i], simplex[(i+1)%3])))
        edges.add(edge)

# Create graph from edges
G_dome = nx.Graph()
for i, p in enumerate(points):
    G_dome.add_node(i, pos=p)
G_dome.add_edges_from(edges)

# Plotting
node_xyz = np.array([G_dome.nodes[i]['pos'] for i in G_dome.nodes()])
edge_xyz = []

for edge in G_dome.edges():
    x0, y0, z0 = G_dome.nodes[edge[0]]['pos']
    x1, y1, z1 = G_dome.nodes[edge[1]]['pos']
    edge_xyz.append(((x0, x1, None), (y0, y1, None), (z0, z1, None)))

edge_trace = go.Scatter3d(
    x=[x for segment in edge_xyz for x in segment[0]],
    y=[y for segment in edge_xyz for y in segment[1]],
    z=[z for segment in edge_xyz for z in segment[2]],
    mode='lines',
    line=dict(color='lightblue', width=1),
    hoverinfo='none'
)

node_trace = go.Scatter3d(
    x=node_xyz[:,0], y=node_xyz[:,1], z=node_xyz[:,2],
    mode='markers',
    marker=dict(
        size=4,
        color='darkblue',
        line=dict(width=0.5, color='black')
    ),
    text=[f"Node {i}" for i in G_dome.nodes()],
    name='Dome Nodes'
)

fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(
    title='Geodesic Dome Representation of Brain Network',
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
    ),
    showlegend=False,
    margin=dict(l=0, r=0, t=50, b=0)
)
fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
import networkx as nx
from itertools import combinations

# Parameters
RADIUS = 1
phi = np.linspace(0, np.pi/2, 10)  # vertical (0 to 90° for dome)
theta = np.linspace(0, 2 * np.pi, 20)  # horizontal (0 to 360°)

# Meshgrid in spherical coordinates
phi, theta = np.meshgrid(phi, theta)

# Convert to cartesian coordinates
x = RADIUS * np.sin(phi) * np.cos(theta)
y = RADIUS * np.sin(phi) * np.sin(theta)
z = RADIUS * np.cos(phi)

# Flatten coordinates
nodes = np.vstack((x.ravel(), y.ravel(), z.ravel())).T

# Create triangle-like edges: connect each point with nearby ones (simplified)
G = nx.Graph()
for i, coord in enumerate(nodes):
    G.add_node(i, pos=coord)

# Connect nodes based on distance threshold
THRESHOLD = 0.3
for i, j in combinations(range(len(nodes)), 2):
    dist = np.linalg.norm(nodes[i] - nodes[j])
    if dist < THRESHOLD:
        G.add_edge(i, j)

# Get 3D positions
pos = nx.get_node_attributes(G, 'pos')
edge_x, edge_y, edge_z = [], [], []
for u, v in G.edges():
    x0, y0, z0 = pos[u]
    x1, y1, z1 = pos[v]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_z.extend([z0, z1, None])
# Random simulated activity (could also be from actual data)
activity = np.random.rand(len(G.nodes()))

# Normalize for color scale
activity_scaled = 100 * (activity - np.min(activity)) / (np.max(activity) - np.min(activity))

# Plot nodes with color gradient
node_x, node_y, node_z = zip(*[pos[i] for i in G.nodes()])

fig = go.Figure()

# Edges
fig.add_trace(go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode='lines',
    line=dict(color='gray', width=1),
    name='Connections'
))

# Nodes
fig.add_trace(go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers',
    marker=dict(
        size=4,
        color=activity_scaled,
        colorscale='Inferno',
        colorbar=dict(title='Activity'),
        opacity=0.9
    ),
    name='Brain Regions'
))

fig.update_layout(
    title='3D Geodesic Dome with Simulated Brain Activity',
    margin=dict(l=0, r=0, t=30, b=0),
    showlegend=False
)

fig.show()


# 3D Geodesic Dome Simulation with Cascading Failures
This simulation demonstrates the foundational geometry behind Link Theory by visualizing a geodesic dome structure in 3D space. The dome is constructed using a hemispherical layout of interconnected triangles, mimicking the distributed load-bearing nature of biological neural networks.

Nodes represent signal hubs or micro-regions in the brain.

Edges represent functional connections or co-activations.

The triangular mesh reflects structural resilience — a core hypothesis of Link Theory.

We simulate a localized failure at a randomly chosen node (in red), which then spreads to its immediate neighbors — representing early collapse and cascade under stress. This illustrates how triangular motifs can either contain or propagate dysfunction based on surrounding structure.

This model supports the theory that early motif collapse (especially in tightly connected triangular circuits) can precede larger breakdowns — offering potential for early detection and intervention.

In [ ]:
import numpy as np
import networkx as nx
import plotly.graph_objects as go
from scipy.spatial import Delaunay
import random

# Step 1: Create dome-like structure
phi = np.linspace(0, np.pi / 2, 10)  # upper hemisphere
theta = np.linspace(0, 2 * np.pi, 20)
phi, theta = np.meshgrid(phi, theta)

x = np.sin(phi) * np.cos(theta)
y = np.sin(phi) * np.sin(theta)
z = np.cos(phi)

points = np.vstack((x.flatten(), y.flatten(), z.flatten())).T
points = points[points[:, 2] >= 0]  # keep upper hemisphere

# Step 2: Delaunay triangulation to find edges
tri = Delaunay(points[:, :2])
edges = set()
for simplex in tri.simplices:
    for i in range(3):
        edge = tuple(sorted((simplex[i], simplex[(i+1)%3])))
        edges.add(edge)

# Step 3: Create graph and add cascade logic
G = nx.Graph()
for i, p in enumerate(points):
    G.add_node(i, pos=p, failed=False)
G.add_edges_from(edges)

# Pick a random node to fail
initial_failure = random.choice(list(G.nodes()))
G.nodes[initial_failure]['failed'] = True
cascade_nodes = {initial_failure}

# Step 4: Cascade rule (neighbor fails with 30% chance if connected to failed node)
for _ in range(3):  # simulate 3 rounds of failure
    new_failures = set()
    for node in cascade_nodes:
        for neighbor in G.neighbors(node):
            if not G.nodes[neighbor]['failed'] and random.random() < 0.3:
                G.nodes[neighbor]['failed'] = True
                new_failures.add(neighbor)
    if not new_failures:
        break
    cascade_nodes = new_failures

# Step 5: Prepare 3D plot
node_xyz = np.array([G.nodes[i]['pos'] for i in G.nodes()])
edge_xyz = []
for edge in G.edges():
    x0, y0, z0 = G.nodes[edge[0]]['pos']
    x1, y1, z1 = G.nodes[edge[1]]['pos']
    edge_xyz.append(((x0, x1, None), (y0, y1, None), (z0, z1, None)))

edge_trace = go.Scatter3d(
    x=[x for segment in edge_xyz for x in segment[0]],
    y=[y for segment in edge_xyz for y in segment[1]],
    z=[z for segment in edge_xyz for z in segment[2]],
    mode='lines',
    line=dict(color='lightgray', width=1),
    hoverinfo='none'
)

node_colors = ['red' if G.nodes[i]['failed'] else 'blue' for i in G.nodes()]
node_trace = go.Scatter3d(
    x=node_xyz[:, 0], y=node_xyz[:, 1], z=node_xyz[:, 2],
    mode='markers',
    marker=dict(size=4, color=node_colors, line=dict(width=0.5, color='black')),
    text=[f"Node {i} - {'Failed' if G.nodes[i]['failed'] else 'Active'}" for i in G.nodes()],
    hoverinfo='text'
)

fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(
    title='3D Geodesic Dome with Cascading Failures',
    scene=dict(xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)),
    showlegend=False,
    margin=dict(l=0, r=0, t=40, b=0)
)

fig.show()


In [ ]:
import itertools

# Find all triangles
triangles = [list(triangle) for triangle in nx.enumerate_all_cliques(G) if len(triangle) == 3]

# Function to draw triangles in 3D
def get_triangle_lines(triangles, pos):
    x_lines, y_lines, z_lines = [], [], []
    for tri in triangles:
        for i, j in itertools.combinations(tri + [tri[0]], 2):
            x_lines += [pos[i][0], pos[j][0], None]
            y_lines += [pos[i][1], pos[j][1], None]
            z_lines += [pos[i][2], pos[j][2], None]
    return x_lines, y_lines, z_lines

# Get triangle lines
x_tri, y_tri, z_tri = get_triangle_lines(triangles, pos)

# Trace for triangle motifs
triangle_trace = go.Scatter3d(
    x=x_tri, y=y_tri, z=z_tri,
    mode='lines',
    line=dict(color='crimson', width=4),
    name='Triangle Motifs'
)

# Combine all in 3D plot
fig = go.Figure(data=[edge_trace, node_trace, triangle_trace])
fig.update_layout(
    title='Triangle Motifs in 3D Geodesic Network',
    scene=dict(xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)),
    margin=dict(l=0, r=0, t=50, b=0),
    showlegend=True
)
fig.show()


In [ ]:
import networkx as nx
import plotly.graph_objects as go
import numpy as np

# Create a synthetic geodesic-like network (a spherical layout)
G = nx.icosahedral_graph()

# 3D spherical layout for geodesic structure
phi = np.linspace(0, np.pi, len(G.nodes()))
theta = np.linspace(0, 2 * np.pi, len(G.nodes()))
x = np.sin(phi) * np.cos(theta)
y = np.sin(phi) * np.sin(theta)
z = np.cos(phi)

pos = {i: (x[i], y[i], z[i]) for i in range(len(G.nodes()))}

# Extract edges
edge_x, edge_y, edge_z = [], [], []
for u, v in G.edges():
    x0, y0, z0 = pos[u]
    x1, y1, z1 = pos[v]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_z.extend([z0, z1, None])

# Plot edges
edge_trace = go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode='lines',
    line=dict(color='lightgray', width=2),
    hoverinfo='none'
)

# Plot nodes
node_x, node_y, node_z = zip(*[pos[i] for i in G.nodes()])
node_trace = go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers+text',
    text=[str(i) for i in G.nodes()],
    marker=dict(
        size=6,
        color='royalblue',
        opacity=0.8
    ),
    textposition="top center",
    hoverinfo='text'
)

# Final figure
fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(
    title='Geodesic Motif Network (Icosahedral Approximation)',
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False)
    ),
    margin=dict(l=0, r=0, t=50, b=0),
    showlegend=False
)
fig.show()


### Triangle Motif Highlighting in 3D

This 3D simulation maps triangle motifs within the geodesic brain-like structure. In Link Theory, triangle motifs are fundamental units of stable signal circulation. Unlike linear or square patterns, triangles resist collapse due to their structural efficiency and closed-loop feedback.

The highlighted crimson lines trace each triangle in the network, representing paths of repeated or reinforced neuronal firing — the backbone of neuroplasticity and, potentially, resilience.


###3D Stress Cascade Simulation

In [ ]:
# Generate 3D positions using spring layout
pos_3d = nx.spring_layout(G, dim=3, seed=42)  # 3D layout

# Extract 3D coordinates
x_nodes = [pos_3d[i][0] for i in G.nodes()]
y_nodes = [pos_3d[i][1] for i in G.nodes()]
z_nodes = [pos_3d[i][2] for i in G.nodes()]

# Create edge traces for 3D visualization
edge_trace = go.Scatter3d(
    x=[pos_3d[edge[0]][0] for edge in G.edges()] + [None],
    y=[pos_3d[edge[0]][1] for edge in G.edges()] + [None],
    z=[pos_3d[edge[0]][2] for edge in G.edges()] + [None],
    mode='lines',
    line=dict(color='gray', width=1),
    hoverinfo='none'
)


In [ ]:
import numpy as np

# Choose a stressed node
stressed_node = 0

# Calculate shortest path lengths from stressed node
stress_levels = nx.single_source_shortest_path_length(G, stressed_node)

# Normalize stress to 0-1 scale (inverse of distance)
max_distance = max(stress_levels.values())
node_stress = {node: 1 - (dist / max_distance) for node, dist in stress_levels.items()}

# Update node colors and sizes based on stress
stress_colors = [f'rgba(255, 0, 0, {node_stress.get(i, 0):.2f})' for i in G.nodes()]
stress_sizes = [5 + 15 * node_stress.get(i, 0) for i in G.nodes()]

node_trace_stress = go.Scatter3d(
    x=x_nodes, y=y_nodes, z=z_nodes,
    mode='markers',
    marker=dict(
        size=stress_sizes,
        color=stress_colors,
        line=dict(width=0.5, color='black')
    ),
    text=[f"Node {i}<br>Stress: {node_stress.get(i, 0):.2f}" for i in G.nodes()],
    name='Stress Levels'
)

fig = go.Figure(data=[edge_trace, node_trace_stress])
fig.update_layout(
    title='3D Stress Propagation from a Single Node',
    scene=dict(xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)),
    margin=dict(l=0, r=0, t=50, b=0),
    showlegend=False
)
fig.show()


In [ ]:
# Find triangle motifs
triangles = [list(c) for c in nx.enumerate_all_cliques(G) if len(c) == 3]

# Choose a triangle to fail (first one)
failed_triangle = triangles[0]
print("Failed Triangle Nodes:", failed_triangle)


Failed Triangle Nodes: [0, 1, 5]


In [ ]:
# Copy graph to simulate failure
G_failed = G.copy()

# Simulate node failure (remove one node of triangle)
failed_node = failed_triangle[0]
G_failed.remove_node(failed_node)


In [ ]:
# Recalculate layout after failure
pos_failed = nx.spring_layout(G_failed, dim=3, seed=42)

x_failed = [pos_failed[i][0] for i in G_failed.nodes()]
y_failed = [pos_failed[i][1] for i in G_failed.nodes()]
z_failed = [pos_failed[i][2] for i in G_failed.nodes()]

# Edge trace
edge_trace_failed = go.Scatter3d(
    x=[pos_failed[edge[0]][0] for edge in G_failed.edges()] + [None],
    y=[pos_failed[edge[0]][1] for edge in G_failed.edges()] + [None],
    z=[pos_failed[edge[0]][2] for edge in G_failed.edges()] + [None],
    mode='lines',
    line=dict(color='black', width=1),
    hoverinfo='none'
)

# Node trace (stress = distance from triangle center)
center_node = failed_triangle[1]
path_lengths = nx.single_source_shortest_path_length(G_failed, center_node)
max_len = max(path_lengths.values())

stress_colors_failed = [f'rgba(255, 100, 100, {1 - (path_lengths.get(i, max_len)/max_len):.2f})' for i in G_failed.nodes()]
stress_sizes_failed = [6 + 10 * (1 - (path_lengths.get(i, max_len)/max_len)) for i in G_failed.nodes()]

node_trace_failed = go.Scatter3d(
    x=x_failed, y=y_failed, z=z_failed,
    mode='markers',
    marker=dict(
        size=stress_sizes_failed,
        color=stress_colors_failed,
        line=dict(width=0.5, color='black')
    ),
    text=[f"Node {i}" for i in G_failed.nodes()],
    name='Post-Failure'
)

# Plot
fig = go.Figure(data=[edge_trace_failed, node_trace_failed])
fig.update_layout(
    title='Simulated Cascade from Triangle Motif Failure',
    scene=dict(xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)),
    margin=dict(l=0, r=0, t=50, b=0),
    showlegend=False
)
fig.show()


### Localized Triangle Motif Failure and Cascade

In this simulation, we removed one node from a highly stable triangle motif to model localized failure — mimicking what happens when a brain region (like the amygdala or hippocampus) collapses due to chronic stress or trauma.

The 3D network plot shows how this failure disrupts nearby nodes and forces the network to reroute, increasing stress (red intensity and node size) as signals take longer, more complex paths. This visually demonstrates **how micro-failures can initiate a system-wide cascade**, a central principle of Link Theory.

Tracking these failures helps anticipate broader collapse and opens new strategies for early intervention.


In [ ]:
# Find triangle motifs (cliques of size 3)
triangles = [clique for clique in nx.find_cliques(G) if len(clique) == 3]

# Visualize just the first few triangles
for tri in triangles[:10]:  # Limit for clarity
    tri_coords = [pos[n] for n in tri] + [pos[tri[0]]]  # close loop
    x_tri, y_tri, z_tri = zip(*tri_coords)
    fig.add_trace(go.Scatter3d(
        x=x_tri, y=y_tri, z=z_tri,
        mode='lines',
        line=dict(color='red', width=3),
        name='Triangle Motif',
        showlegend=False
    ))

fig.show()


In [ ]:
# Pick a "stressed" node
stressed_node = 10
neighbors = list(G.neighbors(stressed_node))

# Reduce its activity to near 0
activity_scaled[stressed_node] = 0

# Slightly increase neighbor activity
for n in neighbors:
    activity_scaled[n] += 10

# Update node visualization
fig.data[1].marker.color = activity_scaled
fig.show()


### Link Theory 3D Dome Simulation Summary

- A hemispheric geodesic dome was constructed using spherical coordinates.
- Network edges were created using spatial proximity, forming biologically inspired connections.
- Activity levels were simulated per region and visualized using an inferno color scale.
- Triangle motifs — fundamental to Link Theory — were identified and visualized.
- Localized failure was introduced by suppressing activity in a single node and simulating compensatory rerouting in neighboring nodes.

This demonstrates the structural logic of Link Theory in 3D with interpretable geometric and topological features.


 # EEG to Dome Mapping

In [ ]:
import pandas as pd

# Load EEG data
df = pd.read_csv('/content/EEG.machinelearing_data_BRMH.csv')  # adjust path if needed

# Preview columns
df.columns[:30]


Index(['no.', 'sex', 'age', 'eeg.date', 'education', 'IQ', 'main.disorder',
       'specific.disorder', 'AB.A.delta.a.FP1', 'AB.A.delta.b.FP2',
       'AB.A.delta.c.F7', 'AB.A.delta.d.F3', 'AB.A.delta.e.Fz',
       'AB.A.delta.f.F4', 'AB.A.delta.g.F8', 'AB.A.delta.h.T3',
       'AB.A.delta.i.C3', 'AB.A.delta.j.Cz', 'AB.A.delta.k.C4',
       'AB.A.delta.l.T4', 'AB.A.delta.m.T5', 'AB.A.delta.n.P3',
       'AB.A.delta.o.Pz', 'AB.A.delta.p.P4', 'AB.A.delta.q.T6',
       'AB.A.delta.r.O1', 'AB.A.delta.s.O2', 'AB.B.theta.a.FP1',
       'AB.B.theta.b.FP2', 'AB.B.theta.c.F7'],
      dtype='object')

In [ ]:
import pandas as pd

# Upload your EEG dataset (make sure the file is uploaded in your runtime)
df = pd.read_csv("EEG.machinelearing_data_BRMH.csv")

# Extract delta-related columns
delta_cols = [col for col in df.columns if 'delta' in col.lower()]
delta_data = df[delta_cols].iloc[0]  # Use data from the first subject/session

# Normalize delta values
delta_norm = (delta_data - delta_data.min()) / (delta_data.max() - delta_data.min())


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import networkx as nx
from scipy.spatial import Delaunay

# Load your EEG dataset
# If not already done, read your file:
# df = pd.read_csv("EEG.machinelearing_data_BRMH.csv")

# STEP 1: Extract delta-related columns
delta_cols = [col for col in df.columns if 'delta' in col.lower()]
delta_data = df[delta_cols].iloc[0]  # Use the first subject/session

# STEP 2: Normalize EEG values
delta_norm = (delta_data - delta_data.min()) / (delta_data.max() - delta_data.min())

# STEP 3: Create node positions for a dome
num_nodes = len(delta_cols)
phi = np.linspace(0, np.pi / 2, num_nodes)  # Dome = hemisphere
theta = np.linspace(0, 2 * np.pi, num_nodes)
x = np.sin(phi) * np.cos(theta)
y = np.sin(phi) * np.sin(theta)
z = np.cos(phi)
positions = np.vstack((x, y, z)).T

# STEP 4: Build graph using Delaunay triangulation
tri = Delaunay(positions)
edges = set()
for simplex in tri.simplices:
    for i in range(3):
        for j in range(i + 1, 3):
            edges.add(tuple(sorted((simplex[i], simplex[j]))))

G = nx.Graph()
for i, coord in enumerate(positions):
    G.add_node(i, pos=coord, activity=delta_norm.iloc[i])
for i, j in edges:
    G.add_edge(i, j)

# STEP 5: Extract node and edge coordinates
node_x, node_y, node_z, node_colors = [], [], [], []
for node in G.nodes():
    x, y, z = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)
    node_z.append(z)
    node_colors.append(G.nodes[node]['activity'])

edge_x, edge_y, edge_z = [], [], []
for i, j in G.edges():
    x0, y0, z0 = G.nodes[i]['pos']
    x1, y1, z1 = G.nodes[j]['pos']
    edge_x += [x0, x1, None]
    edge_y += [y0, y1, None]
    edge_z += [z0, z1, None]

# STEP 6: Plot EEG on 3D geodesic dome
fig = go.Figure()

# Edges
fig.add_trace(go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode='lines',
    line=dict(color='lightgray', width=1),
    name='Connections'
))

# Nodes
fig.add_trace(go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers',
    marker=dict(
        size=6,
        color=node_colors,
        colorscale='Plasma',
        colorbar=dict(title='Delta Activity'),
        opacity=0.9
    ),
    name='EEG Nodes'
))

fig.update_layout(
    title='EEG Delta Activity Mapped onto Geodesic Dome (Link Theory)',
    margin=dict(l=0, r=0, t=40, b=0),
    showlegend=False,
    scene=dict(aspectmode='data')
)

fig.show()


 EEG Mapped onto Link Theory Dome

This visualization maps real EEG Delta wave intensity onto a geodesic dome structure that represents cortical organization.

- Each node represents a brain region/electrode (e.g., FP1, Fz, O2).
- Higher delta power (often related to deep sleep or dysfunction) lights up nodes with higher intensity.
- This allows us to compare real data against motif structures and simulate potential cascade scenarios.


# Simulate Failure of a Key Node

In [ ]:
!pip install trimesh plotly


In [ ]:
import time
import networkx as nx
import numpy as np

# STEP 1: Choose the most active node (simulated failure origin)
origin_node = max(G.nodes, key=lambda n: G.nodes[n].get('activity', 0))

# STEP 2: Identify triangle motifs
triangles = [list(tri) for tri in nx.enumerate_all_cliques(G) if len(tri) == 3]

# Build triangle adjacency mapping
triangle_map = {origin_node: set()}
for tri in triangles:
    if origin_node in tri:
        triangle_map[origin_node].update(set(tri) - {origin_node})

# Now find next layer (2-hop)
triangle_map['layer2'] = set()
for n1 in triangle_map[origin_node]:
    for tri in triangles:
        if n1 in tri:
            triangle_map['layer2'].update(set(tri) - {n1})
triangle_map['layer2'] -= triangle_map[origin_node]
triangle_map['layer2'].discard(origin_node)

# STEP 3: Simulate decay (activity drop) and recolor
def decay_node_colors(failed_nodes):
    node_colors = []
    for node in G.nodes():
        if node in failed_nodes:
            G.nodes[node]['activity'] = 0
        activity = G.nodes[node].get('activity', 0)
        node_colors.append(activity * 100)
    return node_colors

# STEP 4: Visualize at each stage
def plot_step(step_title, failed_nodes):
    node_colors = decay_node_colors(failed_nodes)

    fig = go.Figure()

    # Edges
    fig.add_trace(go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='gray', width=1),
        name='Connections'
    ))

    # Nodes
    fig.add_trace(go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers',
        marker=dict(
            size=5,
            color=node_colors,
            colorscale='Plasma',
            colorbar=dict(title='EEG Intensity (Delta)'),
            opacity=0.9
        ),
        name='EEG Mapped Nodes'
    ))

    fig.update_layout(
        title=step_title,
        margin=dict(l=0, r=0, t=30, b=0),
        showlegend=False
    )

    fig.show()

# STEP 5: Trigger failure cascade
plot_step("Step 1: Origin Node Fails", [origin_node])
plot_step("Step 2: Motif Neighbors Decay", list(triangle_map[origin_node]))
plot_step("Step 3: Second Layer Collapse", list(triangle_map['layer2']))


# Cascade Failure due to one motif

In [15]:
import numpy as np
import networkx as nx
import plotly.graph_objects as go

# Create geodesic dome structure using NetworkX
G = nx.icosahedral_graph()
for _ in range(2):  # Subdivide for more nodes
    G = nx.triangular_lattice_graph(5, 5, periodic=False)
    G = nx.convert_node_labels_to_integers(G)

# Use spring layout for 3D node positioning
pos = nx.spring_layout(G, dim=3, seed=42)

# Map EEG data to nodes (limit to available delta values)
mapped_nodes = list(G.nodes())[:len(delta_norm)]
for node, value in zip(mapped_nodes, delta_norm):
    G.nodes[node]['activity'] = value

# Extract node positions
node_x, node_y, node_z, node_colors = [], [], [], []
for node in G.nodes():
    x, y, z = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_z.append(z)
    activity = G.nodes[node].get('activity', 0)
    node_colors.append(activity * 100)  # Scale for visualization

# Extract edge positions
edge_x, edge_y, edge_z = [], [], []
for u, v in G.edges():
    x0, y0, z0 = pos[u]
    x1, y1, z1 = pos[v]
    edge_x += [x0, x1, None]
    edge_y += [y0, y1, None]
    edge_z += [z0, z1, None]

# Plot the dome
fig = go.Figure()

# Edges
fig.add_trace(go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode='lines',
    line=dict(color='gray', width=1),
    name='Connections'
))

# Nodes
fig.add_trace(go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers',
    marker=dict(
        size=5,
        color=node_colors,
        colorscale='Plasma',
        colorbar=dict(title='EEG Intensity (Delta)'),
        opacity=0.9
    ),
    name='EEG Mapped Nodes'
))

fig.update_layout(
    title='EEG Mapped onto Link Theory Geodesic Dome',
    margin=dict(l=0, r=0, t=40, b=0),
    scene=dict(aspectmode='data'),
    showlegend=False
)

fig.show()


In [16]:
# STEP 1: Find most active node
origin_node = max(G.nodes, key=lambda n: G.nodes[n].get('activity', 0))

# STEP 2: Create failure propagation
cascade_nodes = set([origin_node])
new_nodes = set([origin_node])
depth = 3  # how far cascade spreads

for _ in range(depth):
    next_wave = set()
    for node in new_nodes:
        neighbors = list(G.neighbors(node))
        next_wave.update(neighbors)
    new_nodes = next_wave - cascade_nodes
    cascade_nodes.update(new_nodes)

# STEP 3: Color nodes by cascade level
node_colors_cascade = []
for node in G.nodes():
    if node == origin_node:
        node_colors_cascade.append(1.0)
    elif node in cascade_nodes:
        node_colors_cascade.append(0.6)
    else:
        node_colors_cascade.append(0.1)

# STEP 4: Plot with cascade failure
fig = go.Figure()

# Edges
fig.add_trace(go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode='lines',
    line=dict(color='lightgray', width=1),
    name='Connections'
))

# Nodes with cascade color
fig.add_trace(go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers',
    marker=dict(
        size=5,
        color=node_colors_cascade,
        colorscale='YlOrRd',
        colorbar=dict(title='Cascade Activation'),
        opacity=0.9
    ),
    name='Cascade Effect'
))

fig.update_layout(
    title='Cascade Failure Simulation on Geodesic Dome',
    margin=dict(l=0, r=0, t=40, b=0),
    scene=dict(aspectmode='data'),
    showlegend=False
)

fig.show()


## EEG Mapped onto Link Theory Geodesic Dome

This simulation visualizes real EEG delta wave intensity mapped onto a geodesic dome representing simplified cortical organization. Each node corresponds to a brain region (e.g., FP1, Fz, O2), and color intensity indicates the level of delta wave power.

We then simulate a localized node failure — modeled as the most active (stressed) node — and track how the failure cascades through neighboring nodes over multiple hops. This mimics a breakdown in neural circuits, helping us visualize motif-level collapse within the Link Theory framework.
